# Router

Reference link: https://docs.llamaindex.ai/en/stable/examples/low_level/router.html

To build a router, we’ll walk through the following steps:

- Crafting an initial prompt to select a set of choices
- Enforcing structured output (for text completion endpoints)
- Try integrating with a native function calling endpoint.


In [9]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv("../secrets.env")) # read local .env file

import warnings
warnings.filterwarnings("ignore")


OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

In [10]:
from dataclasses import dataclass, fields
from pydantic import BaseModel, Field, validator
import json

from langchain_community.utils.openai_functions import (
    convert_pydantic_to_openai_function,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

In [11]:
model = ChatOpenAI(temperature=0, api_key=OPENAI_API_KEY)

In [12]:
router_prompt = """
    Eres un router, tu tarea es tomar una decisión entre las siguientes tareas basandote en el mensaje humano:

    "GENERATE" Toma este camino si el mensaje humano requiere de generar un documento legal.
    "ANALIZE" Toma este camino si el mensaje humano requiere de analizar un documento legal.
    "OTHER" Toma este camino si el mensaje humano no coincide con ninguna de las tareas pero se refiere a una tarea legal.
    "INVALID" Toma este camino si el mensaje humano no tiene que ver con ninguna tarea legal.

    Regla 1: Nunca debes inferir información si no aparece en el contexto de la consulta.
    Regla 2: Solo puedes responder con el tipo de consulta que elijas basándote en la razón por la que la elijas.

    Responde solo con el tipo de consulta que elijas, solo una palabra.
    """

In [13]:
prompt = ChatPromptTemplate.from_messages(
    [("system", router_prompt), 
     ("user", "{input}")]
)

In [14]:
@dataclass
class Answer(BaseModel):
    task: str = Field(description="name of the task")
    reason: str = Field(description="reason why it was choosen")

openai_functions = [convert_pydantic_to_openai_function(Answer)]

In [8]:
parser = JsonOutputFunctionsParser()

In [32]:
chain = prompt | model.bind(functions=openai_functions) | parser

In [33]:
chain.invoke({"input": "Genera un contrato de arrendamiento"})

{'task': 'GENERATE',
 'reason': 'El mensaje humano requiere generar un documento legal'}

In [34]:
chain.invoke({"input": "Analiza la siguiente demanda"})

{'task': 'ANALIZE',
 'reason': 'El mensaje humano menciona específicamente la tarea de analizar un documento legal.'}

In [35]:
chain.invoke({"input": "Cuantas horas tiene un mes?"})

{'task': 'INVALID',
 'reason': 'La consulta no está relacionada con una tarea legal.'}